### Exercise 02 : Join

In [1]:
import pandas as pd
import sqlite3

* create a connection to the database using the library sqlite3

In [2]:
conn = sqlite3.connect('../data/checking-logs.sqlite')
cursor = conn.cursor()

In [3]:
preview = pd.io.sql.read_sql('SELECT * FROM checker LIMIT 15', conn)
preview

,index,status,success,timestamp,numTrials,labname,uid
0,0,checking,0,2020-04-16 21:12:50.740474,5,None,admin_1
1,1,ready,0,2020-04-16 21:12:54.708365,5,code_rvw,admin_1
2,2,checking,0,2020-04-16 21:46:47.769088,7,None,admin_1
3,3,ready,0,2020-04-16 21:46:48.121217,7,lab02,admin_1
4,4,checking,0,2020-04-16 21:53:01.862637,6,code_rvw,admin_1
5,5,ready,0,2020-04-16 21:53:05.373389,6,code_rvw,admin_1
6,6,checking,0,2020-04-17 05:18:51.965864,1,None,None
7,7,ready,0,2020-04-17 05:19:02.744528,1,project1,user_4
8,8,checking,0,2020-04-17 05:22:35.249331,2,project1,user_4
9,9,ready,1,2020-04-17 05:22:45.549397,2,project1,user_4


In [4]:
preview = pd.io.sql.read_sql('SELECT * FROM pageviews LIMIT 15', conn)
preview

,index,uid,datetime
0,0,admin_1,2020-04-17 12:01:08.463179
1,1,admin_1,2020-04-17 12:01:23.743946
2,2,admin_3,2020-04-17 12:17:39.287778
3,3,admin_3,2020-04-17 12:17:40.001768
4,4,admin_1,2020-04-17 12:27:30.646665
5,5,admin_1,2020-04-17 12:35:44.884757
6,6,admin_1,2020-04-17 12:35:52.735016
7,7,admin_3,2020-04-17 12:36:21.401412
8,8,admin_3,2020-04-17 12:36:22.023355
9,9,admin_1,2020-04-17 13:55:19.129243


In [5]:
query = """
CREATE TABLE IF NOT EXISTS datamart AS
SELECT
    checker.uid,
    checker.labname, 
    MIN(checker.timestamp) AS first_commit_ts,
    MIN(pageviews.datetime) AS first_view_ts
FROM checker
LEFT JOIN pageviews ON checker.uid = pageviews.uid
WHERE
    checker.status='ready'
    AND checker.numTrials = 1
    AND checker.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
    AND checker.uid LIKE 'user_%'
GROUP BY checker.uid, checker.labname

"""

# First create the table
conn.execute(query)

# Then query the data
datamart = pd.read_sql("SELECT * FROM datamart", conn, parse_dates=['first_commit_ts', 'first_view_ts'])

In [6]:
datamart

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
...,...,...,...,...
135,user_8,laba04s,2020-04-19 10:22:35.761944,NaT
136,user_8,laba05,2020-05-02 13:28:07.705193,NaT
137,user_8,laba06,2020-05-16 17:56:15.755553,NaT
138,user_8,laba06s,2020-05-16 20:01:07.900727,NaT


* using Pandas methods, create two dataframes: test and control
  * test should have the users that have the values in first_view_ts
  * control should have the users that have missing values in first_view_ts
  * replace the missing values in the control with the average first_view_ts of the
  test users, we will use this value for the future analysis
  * save both tables into the database, you will use them in the next exercises
* close the connection

In [7]:
print(datamart['first_view_ts'].dtype)

datetime64[ns]


In [8]:
test = datamart[datamart['first_view_ts'].notna()]
control = datamart[datamart['first_view_ts'].isna()].copy()

mean_view_ts = test['first_view_ts'].mean()
control['first_view_ts'] = control['first_view_ts'].fillna(mean_view_ts)

In [9]:
test.to_sql('test', conn, if_exists='replace', index=False)
control.to_sql('control', conn, if_exists='replace', index=False)

#по сути А/В тестирование, сравниваем две группы пользователей

81

получается проверка гипотезы

```Если студенты видят активность других в ленте (test), они начинают работать над лабораторными раньше (разница между first_commit_ts и дедлайном уменьшается).```

In [10]:
datamart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


In [11]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 0 to 114
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.3+ KB


In [12]:
control.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81 entries, 12 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    81 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 3.2+ KB


In [13]:
test = pd.io.sql.read_sql("SELECT * FROM test", conn)
test

,uid,labname,first_commit_ts,first_view_ts
0,user_1,laba04,2020-04-26 17:06:18.462708,2020-04-26 21:53:59.624136
1,user_1,laba04s,2020-04-26 17:12:11.843671,2020-04-26 21:53:59.624136
2,user_1,laba05,2020-05-02 19:15:18.540185,2020-04-26 21:53:59.624136
3,user_1,laba06,2020-05-17 16:26:35.268534,2020-04-26 21:53:59.624136
4,user_1,laba06s,2020-05-20 12:23:37.289724,2020-04-26 21:53:59.624136
5,user_1,project1,2020-05-14 20:56:08.898880,2020-04-26 21:53:59.624136
6,user_10,laba04,2020-04-25 08:24:52.696624,2020-04-18 12:19:50.182714
7,user_10,laba04s,2020-04-25 08:37:54.604222,2020-04-18 12:19:50.182714
8,user_10,laba05,2020-05-01 19:27:26.063245,2020-04-18 12:19:50.182714
9,user_10,laba06,2020-05-19 11:39:28.885637,2020-04-18 12:19:50.182714


In [14]:
conn.close()